In [1]:
import logging
from copy import deepcopy
from liketable import Liketable
from table_helper import TableHelper
from source_google import GoogleSheetSource
from google_helper import GoogleHelper

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [2]:
# IAM credentials file from Google Console (Service account or OAuth2)
google_creds_file = 'creds.json'

# Replace with your link to google sheets document
# Must be shared with service account client_email
table_url = 'https://docs.google.com/spreadsheets/d/1nLZUKSeqYuskrF5mHOer_BiCWjh84JHHb0BFq53Z2lw/edit?gid=0#gid=0'

# Assume Google Credetials are for Service Acount
# For OAuth2, set client_id and client_secret
gc = GoogleHelper.client_json_creds(filename=google_creds_file, client_id=None, client_secret=None)

# For OAuth2 mode with Google APIs,
# creds.json may need to be updated if refresh token has changed during lifetime.
cb = GoogleHelper.make_file_update_function(google_creds_file)

source = GoogleSheetSource(gc=gc, spreadsheet_url=table_url, refreshtoken_callback=cb)

In [3]:
yandex_token = open('token.txt').read().strip('\n')

w = Liketable(token=yandex_token, language='en')

In [4]:
table_data = source.bulk_read(no_metadata=True)

old_data = deepcopy(table_data)

In [5]:
online_data = w.get_online_data()

2025-07-14 09:30:23,115 - root - INFO - API working...
2025-07-14 09:30:25,341 - root - INFO - Online Likes: artists 103 albums 4 tracks 207


In [6]:
info = w.import_changes(online_data, table_data)

print('Imported likes: ' + ', '.join('%s: %d' % kv for kv in info.items()))

2025-07-14 09:30:25,413 - root - INFO - Likes unset in table from online: 0
2025-07-14 09:30:25,426 - root - INFO - New likes add/set in table: 1


Imported likes: unset: 0, set: 1, new: 0


In [7]:
info = w.upload_changed_likes(online_data, table_data)

print('Uploaded likes: ' + ', '.join('%s: %d' % kv for kv in info.items()))

2025-07-14 09:30:25,511 - root - INFO - API to-do:
2025-07-14 09:30:25,511 - root - INFO - 	Remove like: artists 0 albums 0 tracks 0
2025-07-14 09:30:25,512 - root - INFO - 	Add like:    artists 0 albums 0 tracks 0


Uploaded likes: set: 0, unset: 0


In [8]:
if old_data:
    source.bulk_update(table_data, cached_old_data=old_data)
    print('Google sheets data updated')

Google sheets data updated


In [9]:
if not old_data:
    table_data = TableHelper.sort(table_data)
    source.bulk_write(table_data)
    old_data = table_data

    print('Google sheets data was re/created with all current likes.')